# Demo notebook for spacy models
SSC, April 2023

This demonstrates usage of the spacy NLP modelling methods. 
If you prepare your data on your google drive you can import and export directly to it.

In [ ]:
# Please ignore this cell: extra install steps that are only executed when running the notebook on Google Colab
# flake8-noqa-cell
import os
if 'google.colab' in str(get_ipython()) and not os.path.isdir('Test_Data'):
    # we're running on colab and we haven't already downloaded the test data
    # first install pinned version of setuptools (latest version doesn't seem to work with this package on colab)
    %pip install setuptools==61 -qqq
    # install the moralization package
    %pip install git+https://github.com/ssciwr/moralization-analyzer.git -qqq

    # download test data sets
    !wget https://github.com/ssciwr/moralization-analyzer/archive/refs/heads/test_data.zip -q
    !mkdir -p data && unzip -qq test_data.zip && mv -f moralization-analyzer-test_data/*_Data ./data/. && rm -rf moralization-analyzer-test_data test_data.zip
    !spacy download de_core_news_sm
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
from moralization import DataManager, SpacyModelManager

### Import training data using DataManager

If you need more information about raised warnings run: <br>
```import logging ``` <br>
```logging.getLogger().setLevel(logging.DEBUG)```

In [ ]:
# merge dict example:
merge_dict = {
    "task1": ["KAT1-Moralisierendes Segment"],
    "task2": ["KAT2-Moralwerte", "KAT2-Subjektive Ausdrücke"],
    "task3": ["KAT3-Rolle", "KAT3-Gruppe", "KAT3-own/other"],
    "task4": ["KAT4-Kommunikative Funktion"],
    "task5": ["KAT5-Forderung explizit"],
    "task1+2":["KAT1-Moralisierendes Segment","KAT2-Moralwerte", "KAT2-Subjektive Ausdrücke" ]
}

You can provide the language in the model initialization, as well as the task that should be trained on. Default language is German, and the default task is task1: Training for labels in category 1. The available tasks are:
```
"task1": ["KAT1-Moralisierendes Segment"]
"task2": ["KAT2-Moralwerte", "KAT2-Subjektive Ausdrücke"]
"task3": ["KAT3-Rolle", "KAT3-Gruppe", "KAT3-own/other"]
"task4": ["KAT4-Kommunikative Funktion"]
"task5": ["KAT5-Forderung explizit"]
```

In [ ]:
# train on small dataset
#data_manager = DataManager("/content/data/Test_Data/XMI_11", merge_dict=merge_dict, task = "task2")
data_manager = DataManager("/home/gwydion/SSC/moralization_data/Test_Data", merge_dict=merge_dict, task = "task1+2")


# if you have data in a language different than German, you 
# can pass the selected language model for the corpus language 
# using the language_model keyword argument
# for a selection of the models, see https://spacy.io/usage/models
# data_manager = DataManager("/content/data/Test_Data/XMI_11", language_model="en_core_web_sm")
# train on full dataset
# data_manager = DataManager("/content/data/All_Data/XMI_11") 

In [ ]:
for title, doc in data_manager.doc_dict.items():
    print(f"  - {title}: {len(doc)} tokens")

### Create a new spacy model using ModelManager

- `model_path`: where the ModelManager will store the model and data
- `base_config_file`: spacy model base config file
  - Base configs can be autogenerated here: https://spacy.io/usage/training#quickstart
  - If not provided a default with `Components = "spancat"` is used
- `overwrite_existing_files`: enable this to delete any existing files in `model_path`

You can provide the language in the model initialization, as well as the task that should be trained on. Default language is German, and the default task is task1: Training for labels in category 1. The available tasks are:
```
"task1": ["KAT1-Moralisierendes Segment"]
"task2": ["KAT2-Moralwerte", "KAT2-Subjektive Ausdrücke"]
"task3": ["KAT3-Rolle", "KAT3-Gruppe", "KAT3-own/other"]
"task4": ["KAT4-Kommunikative Funktion"]
"task5": ["KAT5-Forderung explizit"]
```
You can also directly provide the label of the annotation category in the xmi data. Selecting "sc" will train on all labels (not recommended).

In [ ]:
# my_model = SpacyModelManager("my_model", overwrite_existing_files=True)

# select a different language and task
my_model = SpacyModelManager("my_model", overwrite_existing_files=True, language="en", task="task1+2")
# provide a different config file than the base config file
# my_model = SpacyModelManager("my_model", overwrite_existing_files=True, language="en", task="task2", base_config_file="./config.cfg")

In [ ]:
print(my_model)

### Edit metadata

- `metadata` is a dictionary of metadata for the model
- Initally empty apart from default `pipeline` name and `0.0.0` version
- This metadata will also be used to generate the Model Card on hugging face

In [ ]:
my_model.metadata

In [ ]:
my_model.metadata["name"] = "test_pipeline"
my_model.metadata["version"] = "0.1.0"
my_model.metadata["description"] = "A test pipeline for ModelManager testing purposes"
my_model.metadata["author"] = "Liam Keegan"
my_model.metadata["email"] = "liam@keegan.ch"
my_model.metadata["url"] = "https://github.com/ssciwr/moralization"
my_model.metadata["license"] = "MIT"

In [ ]:
my_model.metadata

In [ ]:
print(my_model)

### Train the model

- `data_manager`: the data to be used for training the model
- `overrides`: an optional dictionary of model config settings to override; otherwise all settings are given in the `config.cfg`

In [ ]:
my_model.train(data_manager, overrides={"training.max_epochs": 100}, check_data_integrity = False)

### Evaluate the model

- `data_manager`: the test data from this data_manager will be used to evaluate the model
- the evaluation is carried out using spaCy's [evaluate](https://spacy.io/api/cli#benchmark-accuracy) command via the [evaluate cli](https://github.com/explosion/spaCy/blob/master/spacy/cli/evaluate.py). 

In [ ]:
my_model.evaluate(data_manager)

### Test the model

In [ ]:
my_model.test("Das Wetter ist schön, das Wetter ist unfair.")

### Publish model to hugging-face

- This requires a hugging-face [User Access Token](https://huggingface.co/docs/hub/security-tokens)
- You can export this to the `HUGGING_FACE_TOKEN` environment variable and just call `publish()`
- Or directly pass the token with `publish(hugging_face_token="abc123")`

In [ ]:
urls = my_model.publish()

In [ ]:
urls

### Load an existing model

An existing model can be loaded from its `model_path`:

In [ ]:
loaded_model = SpacyModelManager("my_model")
print(loaded_model)